In [1]:
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.setAppName('practice_2')
conf.set("spark.dynamicAllocation.enabled", "true")
conf.set("spark.sql.sources.partitionOverwriteMode","dynamic")
conf.set("spark.driver.maxResultSize", "4G")
conf.set("spark.driver.memory", "4G")
conf.set("spark.executor.memory", "4G")
conf.set("spark.driver.allowMultipleContexts", "true")

In [7]:
! hdfs dfs -ls ./all_transactions_table.parquet

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2022-09-06 20:40 all_transactions_table.parquet/_SUCCESS
drwxr-xr-x   - ubuntu hadoop          0 2022-09-06 20:40 all_transactions_table.parquet/date=2022-09-06


In [ ]:
spark = pyspark.sql.SparkSession.builder.config(conf=conf).getOrCreate()

table = spark.read.parquet("/user/ubuntu/practice_1/all_transactions_table.parquet")

In [ ]:
table.printSchema()